# Exercise Solutions: Abstractions

## Solution 1: Storage Abstraction

In [ ]:
from abc import ABC, abstractmethod
from typing import Any
from pathlib import Path
import json

class StorageBackend(ABC):
    @abstractmethod
    def save(self, key: str, data: Any) -> None:
        pass
    
    @abstractmethod
    def load(self, key: str) -> Any:
        pass
    
    @abstractmethod
    def exists(self, key: str) -> bool:
        pass
    
    @abstractmethod
    def delete(self, key: str) -> None:
        pass

class LocalStorage(StorageBackend):
    def __init__(self, base_path: str = "./storage"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
    
    def _get_path(self, key: str) -> Path:
        return self.base_path / f"{key}.json"
    
    def save(self, key: str, data: Any) -> None:
        with open(self._get_path(key), 'w') as f:
            json.dump(data, f)
    
    def load(self, key: str) -> Any:
        with open(self._get_path(key)) as f:
            return json.load(f)
    
    def exists(self, key: str) -> bool:
        return self._get_path(key).exists()
    
    def delete(self, key: str) -> None:
        if self.exists(key):
            self._get_path(key).unlink()

class MemoryStorage(StorageBackend):
    def __init__(self):
        self._data: dict[str, Any] = {}
    
    def save(self, key: str, data: Any) -> None:
        self._data[key] = data
    
    def load(self, key: str) -> Any:
        if key not in self._data:
            raise KeyError(f"Key '{key}' not found")
        return self._data[key]
    
    def exists(self, key: str) -> bool:
        return key in self._data
    
    def delete(self, key: str) -> None:
        if key in self._data:
            del self._data[key]

def get_storage(backend_type: str) -> StorageBackend:
    if backend_type == "local":
        return LocalStorage()
    elif backend_type == "memory":
        return MemoryStorage()
    else:
        raise ValueError(f"Unknown backend: {backend_type}")

# Test
storage = get_storage("memory")
storage.save("key1", {"data": "value"})
assert storage.exists("key1")
assert storage.load("key1") == {"data": "value"}
storage.delete("key1")
assert not storage.exists("key1")
print("✅ Solution 1 works!")

## Solution 2: Data Validator Abstraction

In [ ]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class Validator(Protocol):
    def validate(self, data: Any) -> bool: ...
    def get_errors(self) -> list[str]: ...

class SchemaValidator:
    def __init__(self, required_keys: list[str]):
        self.required_keys = set(required_keys)
        self.errors: list[str] = []
    
    def validate(self, data: dict) -> bool:
        self.errors = []
        missing = self.required_keys - set(data.keys())
        if missing:
            self.errors = [f"Missing required keys: {missing}"]
            return False
        return True
    
    def get_errors(self) -> list[str]:
        return self.errors

class RangeValidator:
    def __init__(self, min_val: float, max_val: float):
        self.min_val = min_val
        self.max_val = max_val
        self.errors: list[str] = []
    
    def validate(self, data: float) -> bool:
        self.errors = []
        if not (self.min_val <= data <= self.max_val):
            self.errors = [f"Value {data} not in range [{self.min_val}, {self.max_val}]"]
            return False
        return True
    
    def get_errors(self) -> list[str]:
        return self.errors

class CompositeValidator:
    def __init__(self, validators: list[Validator]):
        self.validators = validators
        self.errors: list[str] = []
    
    def validate(self, data: Any) -> bool:
        self.errors = []
        all_valid = True
        for validator in self.validators:
            if not validator.validate(data):
                all_valid = False
                self.errors.extend(validator.get_errors())
        return all_valid
    
    def get_errors(self) -> list[str]:
        return self.errors

# Test
schema_val = SchemaValidator(required_keys=["name", "age"])
assert schema_val.validate({"name": "Alice", "age": 30})
assert not schema_val.validate({"name": "Bob"})
print(f"Schema errors: {schema_val.get_errors()}")

range_val = RangeValidator(min_val=0, max_val=100)
assert range_val.validate(50)
assert not range_val.validate(150)
print(f"Range errors: {range_val.get_errors()}")

print("✅ Solution 2 works!")

## Solution 3: Cache Decorator

In [ ]:
from dataclasses import dataclass, field

@dataclass
class CachedStorage:
    backend: StorageBackend
    _cache: dict[str, Any] = field(default_factory=dict, init=False)
    _hits: int = field(default=0, init=False)
    _misses: int = field(default=0, init=False)
    
    def save(self, key: str, data: Any) -> None:
        self.backend.save(key, data)
        # Invalidate cache
        if key in self._cache:
            del self._cache[key]
    
    def load(self, key: str) -> Any:
        if key in self._cache:
            self._hits += 1
            return self._cache[key]
        
        self._misses += 1
        data = self.backend.load(key)
        self._cache[key] = data
        return data
    
    def exists(self, key: str) -> bool:
        return key in self._cache or self.backend.exists(key)
    
    def delete(self, key: str) -> None:
        self.backend.delete(key)
        if key in self._cache:
            del self._cache[key]
    
    def cache_stats(self) -> dict[str, int]:
        return {"hits": self._hits, "misses": self._misses}

# Test
base_storage = MemoryStorage()
cached = CachedStorage(base_storage)

cached.save("key1", "value1")
result1 = cached.load("key1")  # Miss
result2 = cached.load("key1")  # Hit

stats = cached.cache_stats()
print(f"Cache stats: {stats}")
assert stats["hits"] == 1
assert stats["misses"] == 1

print("✅ Solution 3 works!")

## Solution 4: Plugin System

In [ ]:
from abc import ABC, abstractmethod
from typing import Any

class Transformer(ABC):
    @property
    @abstractmethod
    def name(self) -> str:
        pass
    
    @abstractmethod
    def transform(self, data: Any) -> Any:
        pass

class TransformerRegistry:
    def __init__(self):
        self._transformers: dict[str, Transformer] = {}
    
    def register(self, transformer: Transformer) -> None:
        self._transformers[transformer.name] = transformer
    
    def get(self, name: str) -> Transformer:
        return self._transformers[name]
    
    def create_pipeline(self, transformer_names: list[str]):
        transformers = [self.get(name) for name in transformer_names]
        return TransformerPipeline(transformers)

class TransformerPipeline:
    def __init__(self, transformers: list[Transformer]):
        self.transformers = transformers
    
    def transform(self, data: Any) -> Any:
        for transformer in self.transformers:
            data = transformer.transform(data)
        return data

class UppercaseTransformer(Transformer):
    @property
    def name(self) -> str:
        return "uppercase"
    
    def transform(self, data: list[str]) -> list[str]:
        return [item.upper() for item in data]

class FilterNullsTransformer(Transformer):
    @property
    def name(self) -> str:
        return "filter_nulls"
    
    def transform(self, data: list) -> list:
        return [item for item in data if item is not None]

# Test
registry = TransformerRegistry()
registry.register(UppercaseTransformer())
registry.register(FilterNullsTransformer())

pipeline = registry.create_pipeline(["filter_nulls", "uppercase"])
result = pipeline.transform(["hello", None, "world"])
print(f"Result: {result}")
assert result == ["HELLO", "WORLD"]

print("✅ Solution 4 works!")

## Solution 5: Multi-Engine Query Builder

In [ ]:
from abc import ABC, abstractmethod
from typing import Any

class QueryBuilder(ABC):
    def __init__(self, collection: str):
        self.collection = collection
    
    @abstractmethod
    def select(self, fields: list[str]):
        pass
    
    @abstractmethod
    def where(self, condition: Any):
        pass
    
    @abstractmethod
    def build(self) -> Any:
        pass

class SQLQueryBuilder(QueryBuilder):
    def __init__(self, table: str):
        super().__init__(table)
        self._fields: list[str] = []
        self._where_clause: str = ""
    
    def select(self, fields: list[str]):
        self._fields = fields
        return self
    
    def where(self, condition: str):
        self._where_clause = condition
        return self
    
    def build(self) -> str:
        fields_str = ", ".join(self._fields) if self._fields else "*"
        query = f"SELECT {fields_str} FROM {self.collection}"
        if self._where_clause:
            query += f" WHERE {self._where_clause}"
        return query

class MongoQueryBuilder(QueryBuilder):
    def __init__(self, collection: str):
        super().__init__(collection)
        self._projection: dict = {}
        self._filter: dict = {}
    
    def select(self, fields: list[str]):
        self._projection = {field: 1 for field in fields}
        return self
    
    def where(self, condition: dict):
        self._filter = condition
        return self
    
    def build(self) -> dict:
        return {
            "collection": self.collection,
            "filter": self._filter,
            "projection": self._projection
        }

# Test
sql_builder = SQLQueryBuilder("users")
sql_query = sql_builder.select(["name", "age"]).where("age > 18").build()
print(f"SQL: {sql_query}")
assert "SELECT name, age FROM users WHERE age > 18" in sql_query

mongo_builder = MongoQueryBuilder("users")
mongo_query = mongo_builder.select(["name", "age"]).where({"age": {"$gt": 18}}).build()
print(f"Mongo: {mongo_query}")
assert mongo_query["filter"]["age"]["$gt"] == 18

print("✅ Solution 5 works!")